Install k3d tooling instead of docker-compose

``` bash
curl -s https://raw.githubusercontent.com/rancher/k3d/master/install.sh | bash
```

In [1]:
!k3d create --publish 80:80 --workers 2

2019/06/20 12:33:33 Created cluster network with ID fb099b10e86538849ba250bcf64e1acced706b2ce2936f8a3ee88de8f10df699
2019/06/20 12:33:33 Creating cluster [k3s-default]
2019/06/20 12:33:33 Creating server using docker.io/rancher/k3s:v0.5.0...
2019/06/20 12:33:34 Booting 2 workers for cluster k3s-default
2019/06/20 12:33:34 Created worker with ID e1dc2b6703bcb8a5547e66b18b65414fb53a1e640c62b70be0436d3f513efbb9
2019/06/20 12:33:35 Created worker with ID 694646a0bdce24d45c273ddaa0e7fed5e3016c0f7323e4b4bf1e632cd3c90d2e
2019/06/20 12:33:35 SUCCESS: created cluster [k3s-default]
2019/06/20 12:33:35 You can now use the cluster with:

export KUBECONFIG="$(k3d get-kubeconfig --name='k3s-default')"
kubectl cluster-info


In [2]:
!cp "$(k3d get-kubeconfig --name='k3s-default')" ~/.kube/config

Test ingress like this example
 https://github.com/rancher/k3d/issues/11#issuecomment-492822086

In [3]:
%%bash
cat <<EOF | kubectl create -f -
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-demo-dom
  labels:
    app: nginx-demo-dom
spec:
  replicas: 2
  selector:
    matchLabels:
      app: nginx-demo-dom
  template:
    metadata:
      labels:
        app: nginx-demo-dom
    spec:
      containers:
      - name: nginx-demo-dom
        image: nginx:latest
        ports:
        - containerPort: 80
---
apiVersion: v1
kind: Service
metadata:
  name: nginx-demo-dom
  labels:
    app: nginx-demo-dom
spec:
  ports:
    - port: 8081
      targetPort: 80
      name: http
  selector:
    app: nginx-demo-dom
---
apiVersion: extensions/v1beta1
kind: Ingress
metadata:
  name: nginx-demo-dom
  annotations:
    ingress.kubernetes.io/ssl-redirect: "false"
spec:
  rules:
  - host: k3d-ingress-demo.com
    http:
      paths:
      - backend:
          serviceName: nginx-demo-dom
          servicePort: 8081
EOF

deployment.apps/nginx-demo-dom created
service/nginx-demo-dom created
ingress.extensions/nginx-demo-dom created


In [10]:
!kubectl get deployments

NAME             READY   UP-TO-DATE   AVAILABLE   AGE
nginx-demo-dom   2/2     2            2           39s


wait untill available becomes at least 1

In [11]:
!kubectl get svc

NAME             TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)    AGE
kubernetes       ClusterIP   10.43.0.1     <none>        443/TCP    113s
nginx-demo-dom   ClusterIP   10.43.139.7   <none>        8081/TCP   40s


In [12]:
!curl -H "Host: k3d-ingress-demo.com" http://localhost

<!DOCTYPE html>
<html>
<head>
<title>Welcome to nginx!</title>
<style>
    body {
        width: 35em;
        margin: 0 auto;
        font-family: Tahoma, Verdana, Arial, sans-serif;
    }
</style>
</head>
<body>
<h1>Welcome to nginx!</h1>
<p>If you see this page, the nginx web server is successfully installed and
working. Further configuration is required.</p>

<p>For online documentation and support please refer to
<a href="http://nginx.org/">nginx.org</a>.<br/>
Commercial support is available at
<a href="http://nginx.com/">nginx.com</a>.</p>

<p><em>Thank you for using nginx.</em></p>
</body>
</html>


In [13]:
# delete cluster
!k3d d 

2019/06/20 12:35:47 Removing cluster [k3s-default]
2019/06/20 12:35:47 ...Removing 2 workers
2019/06/20 12:35:49 ...Removing server
2019/06/20 12:35:51 SUCCESS: removed cluster [k3s-default]
